In [16]:
import numpy as np
from source import data_import
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [17]:
df = data_import.load_pyarrow(file_name="feature_extracted")

/Users/tassiloheinrich/Documents/DS/ai-seizure-detectives/source/../data/feature_extracted.arrow was loaded.


In [18]:
df.head()

,seizure_id,segment_id,is_seizure,F4-C4_mean,F4-C4_std,F4-C4_var,F4-C4_delta,F4-C4_theta,F4-C4_gamma,F3-C3_mean,...,FP1-F3_var,FP1-F3_delta,FP1-F3_theta,FP1-F3_gamma,CZ-PZ_mean,CZ-PZ_std,CZ-PZ_var,CZ-PZ_delta,CZ-PZ_theta,CZ-PZ_gamma
0,0,1,False,0.642823,25.278278,638.991344,99.724544,33.651173,0.131576,-0.136380,...,704.741392,115.680103,29.233559,0.343249,0.758870,35.447713,1256.540374,194.364620,76.886992,0.174189
1,0,2,False,0.459926,24.908361,620.426452,80.467433,30.739370,0.113581,0.607864,...,654.580741,98.999296,24.519002,0.363930,0.040398,33.861154,1146.577756,187.417290,34.401955,0.181605
2,0,3,False,-0.336786,25.078313,628.921793,97.815139,19.120020,0.108965,0.020483,...,505.928076,76.273119,21.891098,0.347473,0.139786,30.472674,928.583879,117.606520,31.998680,0.155509
3,0,4,False,0.571550,23.796227,566.260437,78.057846,22.859691,0.112154,0.170771,...,623.521637,111.418632,24.584840,0.326450,0.347514,30.803236,948.839359,158.377845,31.605055,0.181355
4,0,5,False,-0.156618,28.030262,785.695596,102.786435,31.697164,0.123064,0.019398,...,838.658633,115.905093,37.751880,0.323275,-0.032786,39.120852,1530.441051,215.122132,60.283108,0.170928


In [19]:
df.shape

(1138, 63)

In [20]:
# Uncomment if you want to try without "segment_id"
#drop_features = ["segment_id"]
#model_df = df.drop(columns=drop_features)

# Uncomment if you want to try with "segment_id"
model_df = df

In [21]:
y = model_df["is_seizure"]
X = model_df.drop(columns=["is_seizure"])

In [22]:
def run_classification_models(X, y, models=None, cv=5, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
    
    if models is None:
        models = {
            'Logistic Regression': {
                'model': LogisticRegression(),
                'params': {
                    'C': [0.001, 0.01, 0.1, 1, 10],
                    'max_iter': [1000, 1500, 2000]
                }
            },
            'Random Forest': RandomForestClassifier(),
            'Gradient Boosting': GradientBoostingClassifier(),
            'XGBoost': XGBClassifier()
            # Add more models here
        }

    for model_name, model_info in models.items():
        print(f"Model: {model_name}")
        
        if isinstance(model_info, dict):
            model = model_info['model']
            param_grid = model_info['params']
            grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
        else:
            model = model_info
            best_model = model
        
        # Train the best model on the training data
        best_model.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = best_model.predict(X_test)
        
        # Calculate and display the confusion matrix
        confusion = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(confusion)
        
        # Generate a classification report with various metrics
        report = classification_report(y_test, y_pred)
        print("Classification Report:")
        print(report)
        print()

# Usage example:
# X and y are your feature matrix and target variable
# run_classification_models(X, y)


In [23]:
run_classification_models(X, y)

Model: Logistic Regression


/Users/tassiloheinrich/Documents/DS/ai-seizure-detectives/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tassiloheinrich/Documents/DS/ai-seizure-detectives/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver 

Confusion Matrix:
[[173   5]
 [  4  46]]
Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.97      0.97       178
        True       0.90      0.92      0.91        50

    accuracy                           0.96       228
   macro avg       0.94      0.95      0.94       228
weighted avg       0.96      0.96      0.96       228


Model: Random Forest
Confusion Matrix:
[[175   3]
 [  5  45]]
Classification Report:
              precision    recall  f1-score   support

       False       0.97      0.98      0.98       178
        True       0.94      0.90      0.92        50

    accuracy                           0.96       228
   macro avg       0.95      0.94      0.95       228
weighted avg       0.96      0.96      0.96       228


Model: Gradient Boosting
Confusion Matrix:
[[176   2]
 [  3  47]]
Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.99      0.99       1